In [1]:
import networkx as nx
import rustworkx as rx
import pandas as pd
import cProfile

In [2]:
def build_drug_interaction_network(graph_builder):
    # Drug-drug interaction network
    # https://snap.stanford.edu/biodata/datasets/10001/10001-ChCh-Miner.html
    data = pd.read_csv(
        "https://snap.stanford.edu/biodata/datasets/10001/files/ChCh-Miner_durgbank-chem-chem.tsv.gz",
        sep="\t",
        header=None,
    )
    return graph_builder(data, source=0, target=1)

# Betweenness Centrality measure (NetworkX)

In [3]:
def profile(profile_out: str = None):
    def wrapper(func):
        def inner(*args, **kwargs):
            import cProfile
            with cProfile.Profile() as pr:
                res = pr.runcall(func, *args, **kwargs)
                pr.dump_stats(f'{profile_out}.profile') if profile_out is not None else pr.print_stats()
                return res
        return inner
    return wrapper

In [4]:
profiler = profile()

In [5]:
g = build_drug_interaction_network(graph_builder=nx.from_pandas_edgelist)
res = profiler(nx.betweenness_centrality)(g)

         51195879 function calls (51195875 primitive calls) in 25.198 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <class 'networkx.utils.decorators.argmap'> compilation 8:1(<module>)
        1    0.000    0.000   25.198   25.198 <class 'networkx.utils.decorators.argmap'> compilation 8:1(argmap_betweenness_centrality_5)
        1    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.157    0.157   25.197   25.197 betweenness.py:14(betweenness_centrality)
     1514   17.814    0.012   21.810    0.014 betweenness.py:256(_single_source_shortest_path_basic)
     1514    3.078    0.002    3.230    0.002 betweenness.py:317(_accumulate_basic)
        1    0.000    0.000    0.000    0.000 betweenness.py:359(_rescale)
        1    0.000    0.000   25.198   25.198 cProfile.py:106(runcall)
  4560216    0.367    0.000    0.367    0.000 coreviews.py:44(__init__)
  228010

`_single_source_shortest_path_basic` gets call for every node in the graph (1514 times).

In [6]:
len(g)

1514

In [7]:
! snakeviz nx_betweenness

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fhome%2Fleonardo%2FEscritorio%2Ffiuba%2FTPF%2Fnx-profiling%2Fnx_betweenness
^C

Bye!


# Betweenness Centrality measure (RustworkX)

In [8]:
def rx_graph_builder(data, *args, **kwargs):
    # the graph is not as easy to build as in networkx...
    nodes = data.stack().reset_index()[0].unique()
    g = rx.PyGraph()
    node_idxs = g.add_nodes_from(nodes)
    node_to_index_mapping = dict(zip(nodes, node_idxs))
    for edge in data.itertuples():
        node_1, node_2 = node_to_index_mapping[edge._1], node_to_index_mapping[edge._2]
        g.add_edge(node_1, node_2, None)
    return g

In [9]:
g = build_drug_interaction_network(graph_builder=rx_graph_builder)

In [10]:
res = profiler(rx.betweenness_centrality)(g)

         9 function calls in 0.313 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.313    0.313 __init__.py:1593(_graph_betweenness_centrality)
        1    0.000    0.000    0.313    0.313 cProfile.py:106(runcall)
        1    0.000    0.000    0.000    0.000 functools.py:818(dispatch)
        1    0.000    0.000    0.313    0.313 functools.py:884(wrapper)
        1    0.000    0.000    0.000    0.000 weakref.py:415(__getitem__)
        1    0.000    0.000    0.000    0.000 weakref.py:428(__setitem__)
        1    0.313    0.313    0.313    0.313 {graph_betweenness_centrality}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'enable' of '_lsprof.Profiler' objects}


